In [1]:
import numpy as np
import pandas as pd
import polars as pl

# 商品ベクトルの作成

- 最終的に作りたいもの
    - 購買履歴順に並んだ商品ベクトル集合と，その集合に対応する目的変数3つのリスト
- 作成手順
    1. 商品ベクトルの作成（word2vecを平均化）
    2. 購買履歴をidで表現（商品ベクトル集合の元），各集合に対応する目的変数のリストを作成
    3. idの購買履歴を，商品ベクトル辞書を元にベクトルに置き換える．
    4. 2と3で定義した入力と出力でLSTM
- 商品ベクトル作成
    とりあえず何かしらをword2vecでベクトルにする
    -   商品名+カテゴリ全粒度を結合
    -   上記をembeddingにする（単語平均化）

## 前処理

In [2]:
product = pd.read_csv("../../../raw_data/cainz_dataset/products/EC商品マスタ_with_pb.csv")
product.head()

/var/folders/j5/73_tgpps3n3_9x3yn27l04nw0000gn/T/ipykernel_38025/809983754.py:1: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  product = pd.read_csv("../../../raw_data/cainz_dataset/products/EC商品マスタ_with_pb.csv")


,jigyoubu,dpt,line,class,category_code,ec_category,product_code,product_name,brand,ec_price
0,01_インテリア・ファニシング,060_収納,602_キッチン収納,6020_ワゴン,c122212,122212：キッチンワゴン・収納庫,4549509633204,◆Ｊ２１キッチンワゴン スリム（ＷＨ）,PB,3980.0
1,01_インテリア・ファニシング,060_収納,602_キッチン収納,6020_ワゴン,c122212,122212：キッチンワゴン・収納庫,4528302261982,リセスタイル テーブルワゴン３段 ホワイト,NB,590.0
2,01_インテリア・ファニシング,060_収納,602_キッチン収納,6020_ワゴン,NaN,NaN,4936695264730,◆Ｎ１３ ワゴン３段ＰＷＡ－４５７０ＩＶ,PB,1980.0
3,01_インテリア・ファニシング,060_収納,602_キッチン収納,6020_ワゴン,c122212,122212：キッチンワゴン・収納庫,4983771988202,レンジラックワイドＤＲＷ－ＷＩＤＥ ＡＩＶ／ＳＷＨ,NB,12800.0
4,01_インテリア・ファニシング,060_収納,602_キッチン収納,6020_ワゴン,c122212,122212：キッチンワゴン・収納庫,4549509612803,◆簡単組立スチールワゴン３段,PB,1980.0


In [3]:
product.sample(10)

,jigyoubu,dpt,line,class,category_code,ec_category,product_code,product_name,brand,ec_price
324663,21_ＰＲＯツール,092_建築工具,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,4537327039704,チューブ継手 ファイブ異径ユニオンティ ８ｍｍ×６ｍｍ（２箇所）,NaN,378.0
197649,21_ＰＲＯツール,075_建築金物,752_壁金具,7523_棚受・棚柱・ダボ,c453014,453014：ＤＩＹ 棚受け,4973658087609,Ａ－３８８Ｓ ＳバーＢＫ角パイプ用左２５０ｍｍ,NB,1180.0
377492,21_ＰＲＯツール,092_建築工具,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,5060063227801,ＬＥＤ投光器 ＯＭＮＩ２Ｋ ２０００ルーメン,NaN,4828.0
224738,21_ＰＲＯツール,092_建築工具,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,4963202098427,Ｌ型クランプ エビ万力 グリップハンドル ２５０ｍｍ×１２１ｍｍ,NaN,6908.0
540642,29_ワーキング,017_ワーキング,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,4988888101599,防寒ジャケット ７２１０－１７－Ｓ クーガー,NaN,10288.0
223229,21_ＰＲＯツール,092_建築工具,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,4989433313146,コンビネーションレンチ１８ｍｍ,NaN,1568.0
511623,24_ＰＲＯファーム,050_園芸ツール,504_散水用品,5047_潅水用品,c542830,542830：散水ノズル,4975373154550,タカギ ９ｍｍジョイントバルブ ＧＫＡ１０３,NB,1180.0
500235,24_ＰＲＯファーム,031_農業資材・機械,314_収穫・出荷資材,3144_出荷資材,c612425,612425：たばねらテープ,4952271161217,テープナー用テープ 青 １０Ｐ,NB,880.0
473301,22_ＰＲＯ資材,027_建築資材,000_ドロップシップ,0000_ドロップシップ,NaN,NaN,4571411263323,ＦＴＡＰ ユニオン溶接型フレキ（フッ素樹脂パッキン付） 継手ステンレス ２０ＡＸ５００Ｌ,NaN,10698.0
82025,06_家電,094_家事・リビング,943_クリーナー,9435_クリーナーパーツ,c332212,332212：紙パック・クリーナーパーツ,4549077602299,紙パック ＡＭＣ－ＮＣ６,NB,1480.0


In [4]:
product.columns

Index(['jigyoubu', 'dpt', 'line', 'class', 'category_code', 'ec_category',
       'product_code', 'product_name', 'brand', 'ec_price'],
      dtype='object')

In [5]:
tmp = product[['jigyoubu', 'dpt', 'line', 'class',"product_name"]].to_numpy()
tmp2 = product["product_code"].values.tolist()
id_name_pair = dict()
for i, row in enumerate(tmp):
    id_name_pair[tmp2[i]] = ''.join(row)
id_name_pair

{4549509633204: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴン◆Ｊ２１キッチンワゴン\u3000スリム（ＷＨ）',
 4528302261982: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンリセスタイル\u3000テーブルワゴン３段\u3000ホワイト',
 4936695264730: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴン◆Ｎ１３\u3000ワゴン３段ＰＷＡ－４５７０ＩＶ',
 4983771988202: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンレンジラックワイドＤＲＷ－ＷＩＤＥ\u3000ＡＩＶ／ＳＷＨ',
 4549509612803: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴン◆簡単組立スチールワゴン３段',
 4962191015866: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンサージュワゴンＷＴ４段ホワイト',
 4549509621423: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴン◆組立簡単ワゴンスリム３段\u3000ＷＨ',
 4549813990512: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンすき間ラック６段３０Ｗ\u3000ホワイト',
 4962191014555: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンスマートワゴンＦｉｔ\u3000ＷＴ２５０－３段',
 4936695315685: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴン◆Ｉ３\u3000キッチンストッカー４段\u3000ブラウン',
 4528302259590: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンリセスタイル\u3000スーパースリムラック\u3000ウッドマルチ４段\u3000ホワイト',
 4560373441238: '01_インテリア・ファニシング060_収納602_キッチン収納6020_ワゴンワゴン４段２０Ｗ',
 49

## word2vec

In [42]:
import MeCab
import re
from gensim.models import KeyedVectors
from tqdm import tqdm
model_dir = '../../entity_vector.model.bin'
model = KeyedVectors.load_word2vec_format(model_dir, binary=True)

id_emb = dict()

for id, text in tqdm(id_name_pair.items()):
    tokens = MeCab.Tagger("-Owakati").parse(text).split()

    # 埋め込みベクトルの初期化（空の2次元配列）
    emb = np.empty((0, 200))  # 200はベクトルの次元数
    for tmp in tokens:
        if tmp in model:  # モデルに単語が含まれているかチェック
            # 単語ベクトルを追加（新たな行として）
            emb = np.vstack([emb, model[tmp]])
        else:
            pass

    id_emb[id] = np.mean(emb, axis = 0)

id_emb

100%|██████████| 560240/560240 [04:18<00:00, 2169.71it/s]


{4549509633204: array([-1.12852518e+00,  8.16079253e-02, -1.17840074e+00, -5.17136835e-01,
         4.09476111e-01,  1.45425851e-02,  3.04010248e-01,  5.42267663e-01,
        -5.58828936e-01, -5.90180055e-01,  2.64585159e-02, -4.89703143e-01,
         1.42884291e-01,  4.30828640e-01,  6.37090865e-01, -1.56145406e-01,
         1.15864034e-01, -5.69461328e-03,  9.00012742e-01,  7.36708416e-01,
         1.01046899e-01,  8.61752127e-02,  1.57366184e-01, -2.66383525e-01,
         9.34885406e-01,  5.74972566e-01, -3.21870738e-01,  2.80401901e-01,
        -5.52275168e-01, -1.19278626e+00,  7.61803268e-01, -1.57408618e-01,
        -5.98862719e-01, -5.37482859e-01,  3.76010281e-01, -3.00989063e-01,
        -1.03777644e+00,  7.10949540e-01,  1.40536857e+00, -1.51194022e+00,
         2.48543887e-01, -6.26731901e-01, -5.81230711e-01,  1.58702247e-02,
         7.50964727e-01,  7.26756649e-01, -4.56375113e-01, -6.29932766e-01,
        -6.95812565e-03,  6.40924502e-01, -7.26487268e-01,  2.36045589e-0

In [43]:
len(id_emb)

560240